In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import pathlib
DATA_DIR = pathlib.Path('/home/HDD6TB/datasets/emotions/ABAW/eccv_4')

In [3]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
import mord

from collections import defaultdict
import os
import glob
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [4]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=8):
    # compute_acc
    acc = accuracy_score(gt,pred)
    # compute_F1
    gt = one_hot_transfer(gt,class_num)
    pred = one_hot_transfer(pred,class_num)
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1

def CCC_score(x, y):
    vx = x - np.mean(x)
    vy = y - np.mean(y)
    rho = np.sum(vx * vy) / (np.sqrt(np.sum(vx**2)) * np.sqrt(np.sum(vy**2)))
    x_m = np.mean(x)
    y_m = np.mean(y)
    x_s = np.std(x)
    y_s = np.std(y)
    ccc = 2*rho*x_s*y_s/(x_s**2 + y_s**2 + (x_m - y_m)**2)
    return ccc

def metric_for_VA(gt_V,gt_A,pred_V,pred_A):
    ccc_V,ccc_A = CCC_score(gt_V,pred_V),CCC_score(gt_A,pred_A)
    return ccc_V,ccc_A, 0.5*(ccc_V+ccc_A)

def CCC_numpy(y_true, y_pred):
    '''Reference numpy implementation of Lin's Concordance correlation coefficient'''
    
    # covariance between y_true and y_pred
    s_xy = np.cov([y_true, y_pred])[0,1]
    # means
    x_m = np.mean(y_true)
    y_m = np.mean(y_pred)
    # variances
    s_x_sq = np.var(y_true)
    s_y_sq = np.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2)
    
    return ccc

In [5]:
compare_filenames=lambda x: int(os.path.splitext(x)[0])

In [6]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, Flatten, LSTM, Activation, MaxPooling2D
from tensorflow.keras.regularizers import l2 as L2_reg
from tensorflow.keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D, Input, Masking, Conv3D, MaxPooling3D, GlobalMaxPool3D
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow.keras.backend as K 
from tensorflow.keras.models import load_model,Model

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

2.4.1


## Feature extraction

In [7]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

print(f"Torch: {torch.__version__}")
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

Torch: 1.7.1+cu110
True


In [8]:
#num_classes=7
if False:
    #PATH='affectnet_vggface2_enet2_gmp_smooth.pt'
    PATH='enet_b2_8.pt'
    #PATH='enet_b2_7.pt'
    IMG_SIZE=260 #224 #
else:
    #PATH='affectnet_vggface2_enet0.pt'
    #PATH='affectnet_vggface2_enet0_new.pt'
    #PATH='enet_b0_7.pt'
    #PATH='enet_b0_8_best_afew.pt'
    
    #PATH='enet_b0_8_best_vgaf.pt'
    PATH='enet_b0_8_va_mtl.pt'
    IMG_SIZE=224
    
#IMG_SIZE=112

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

In [9]:
print(PATH)
feature_extractor_model = torch.load('../../models/affectnet_emotions/'+PATH)

enet_b0_8_va_mtl.pt


In [10]:
if False:
    classifier_weights=feature_extractor_model.classifier[0].weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier[0].bias.cpu().data.numpy()
else:
    classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
    classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(10, 1280) [[ 0.00679121  0.09001139  0.0694934  ...  0.10351563 -0.00995514
  -0.17376047]
 [-0.004085   -0.07134113 -0.12164655 ... -0.05029012  0.03059323
   0.08198261]
 [-0.0034241  -0.06510569 -0.00448079 ...  0.04248156 -0.10535879
  -0.00544419]
 ...
 [ 0.07878461 -0.03540913 -0.06665969 ... -0.10314589  0.1332206
  -0.06119434]
 [-0.02409821 -0.00270485  0.00887998 ... -0.00501176  0.01345664
  -0.02131553]
 [-0.04529824 -0.04590099 -0.00950194 ...  0.00753843  0.02128105
  -0.05743075]]
(10,) [-0.03629377 -0.00268708 -0.05411524  0.01482256  0.13788255  0.09921926
 -0.05259513 -0.0124341   0.0816549   0.04621203]


In [11]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model=feature_extractor_model.to(device)
feature_extractor_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [12]:
def get_probab(features, logits=True):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias
    if logits:
        return x
    #print(x)
    e_x = np.exp(x - np.max(x,axis=1)[:,np.newaxis])
    #print(x.shape,x,np.max(x,axis=1),e_x)
    return e_x / e_x.sum(axis=1)[:,None]

In [12]:
print(test_transforms)
#data_dir=DATA_DIR / 'cropped'

data_dir=DATA_DIR / 'cropped_aligned'
#data_dir=DATA_DIR / 'SR/cropped_aligned'
print(data_dir)
img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=data_dir / filename 
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(data_dir /filename / img_name)
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=64: #96: #48: #32:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

    #X_scores=X_global_features #get_probab(X_global_features)
    #print(X_global_features.shape,X_scores.shape)

Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/eccv_4/SR/cropped_aligned


  0%|          | 0/307 [00:00<?, ?it/s]

In [13]:
X_scores=get_probab(X_global_features,logits=True)
print(X_scores[:5])

[[ 1.8688838e+00 -1.2798675e+00  4.0342087e-01 -1.9031891e+00
  -3.6277884e-01  2.7075288e+00 -4.8229852e-01  1.6048644e+00
  -1.4294155e-02  2.8019422e-01]
 [-1.3015268e+00 -3.5388476e-01 -3.5591192e+00 -7.4799877e-01
   1.4669149e+00  4.0818615e+00 -3.0371362e-01  3.3040609e+00
   1.3480158e-01 -1.0034430e-01]
 [-5.5128229e-01  4.1866582e-04 -2.1932106e+00 -1.8251675e+00
   3.1605902e+00  2.8578134e+00 -1.9422179e+00  2.0182512e+00
   4.4615337e-01 -5.6445170e-02]
 [-4.1735449e-01 -1.7355011e+00 -2.4780661e-01 -1.4196098e-01
   2.2339792e+00  8.8991779e-01 -3.8366860e-01  1.1287602e+00
   4.1341019e-01  2.0222169e-01]
 [ 9.8042774e-01 -3.6756520e+00 -1.9611456e+00  1.1986889e+00
   1.6967728e+00  5.7321155e-01 -1.0199708e+00  3.3609467e+00
   3.7330532e-01  5.6045234e-01]]


In [14]:
filename2featuresAll={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
#filename2featuresAll={img_name:(global_features,) for img_name,global_features, in zip(img_names,X_global_features)}
print(len(filename2featuresAll))

168097


In [ ]:
print(test_transforms)
data_dir=DATA_DIR / 'mtl_test_data/cropped_aligned'
print(data_dir)
img_names=[]
X_global_features=[]
imgs=[]
for filename in tqdm(os.listdir(data_dir)):
    frames_dir=data_dir / filename 
    for img_name in os.listdir(frames_dir):
        if img_name.lower().endswith('.jpg'):
            img = Image.open(data_dir /filename / img_name)
            img_tensor = test_transforms(img)
            if img.size:
                img_names.append(filename+'/'+img_name)
                imgs.append(img_tensor)
                if len(imgs)>=64: #96: #48: #32:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)
                    imgs=[]

if len(imgs)>0:        
    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
    features=features.data.cpu().numpy()

    if len(X_global_features)==0:
        X_global_features=features
    else:
        X_global_features=np.concatenate((X_global_features,features),axis=0)

    imgs=[]

    #X_scores=X_global_features #get_probab(X_global_features)
    #print(X_global_features.shape,X_scores.shape)

Compose(
    Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)
/home/HDD6TB/datasets/emotions/ABAW/eccv_4/mtl_test_data/cropped_aligned


  0%|          | 0/82 [00:00<?, ?it/s]

In [15]:
X_scores=get_probab(X_global_features,logits=True)
print(X_scores[:5])

[[-1.2368399e+00  1.3957083e+00 -2.1751339e+00 -2.6532381e+00
   3.7151897e+00  2.1231325e+00 -1.2529771e+00  1.8865112e+00
   6.5451306e-01 -1.1197664e-03]
 [-4.5890012e-01  1.7170735e-01 -2.1290927e+00 -1.3519958e+00
   2.0608756e-01  2.9360936e+00  9.5479238e-01  3.2599382e+00
   6.9401488e-03 -3.6578562e-02]
 [-9.2518651e-01 -1.5683583e+00 -1.1122453e+00  1.2513974e+00
   9.5606560e-01  1.4045405e+00 -1.0135034e+00  4.0808926e+00
   2.7149817e-01  4.7487086e-01]
 [-6.6330218e-01  3.7657097e-01 -7.0575130e-01 -3.1108387e+00
   2.9681218e+00  3.2157569e+00  3.4092167e-01  5.1639426e-01
   4.2772499e-01 -1.6744725e-01]
 [-1.8718929e+00  2.6364112e+00 -2.2357848e+00 -4.3089094e+00
   4.6268377e+00  2.8123999e+00 -1.0999391e+00  6.6852532e-02
   7.0621473e-01 -1.4183587e-01]]


In [16]:
filename2featuresTest={img_name:(global_features,scores) for img_name,global_features,scores in zip(img_names,X_global_features,X_scores)}
print(len(filename2featuresTest))


50441


In [18]:
with open('enet_b0_8_va_mtl_aligned_test.pickle', 'wb') as handle:
    pickle.dump(filename2featuresTest, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Save/load features 

In [11]:
import pickle
if False:
    model_name='mymobilenet_7_ft_sgd_model'
    num_classes=7
else:
    #model_name='enet0_8'
    num_classes=8
    #model_name='enet0_7'
    #model_name='enet_b0_8_best_afew'
    #model_name='enet_b0_8_best_vgaf' #first three challenges
    #model_name='enet2_8' #MTL challenge
    #model_name='enet2_7'
    model_name='enet_b0_8_va_mtl' 

model_name+='_aligned'#'_SR' #
#MODEL2FEATURES=model_name+'_aligned.pickle'
MODEL2FEATURES=model_name+'.pickle' 

MODEL2FEATURES='../../../emotions-multimodal/faces/ABAW/abaw4/'+MODEL2FEATURES

print(MODEL2FEATURES)

../../../emotions-multimodal/faces/ABAW/abaw4/enet_b0_8_va_mtl_aligned.pickle


In [16]:
if False:
    with open(MODEL2FEATURES, 'wb') as handle:
        pickle.dump(filename2featuresAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open(MODEL2FEATURES, 'rb') as handle:
    filename2featuresAll=pickle.load(handle)
print(len(filename2featuresAll))

168097


In [9]:
with open('enet_b0_8_va_mtl_aligned_test.pickle', 'rb') as handle:
    filename2featuresTest=pickle.load(handle)
print(len(filename2featuresTest))

50441


------------ 

# Classifiers

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.applications import efficientnet as enet
import efficientnet.tfkeras as enet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
from copy import deepcopy
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = deepcopy(self.model.get_weights())

In [15]:
def CCC(y_true, y_pred):
    '''Lin's Concordance correlation coefficient: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    
    The concordance correlation coefficient is the correlation between two variables that fall on the 45 degree line through the origin.
    
    It is a product of
    - precision (Pearson correlation coefficient) and
    - accuracy (closeness to 45 degree line)

    Interpretation:
    - `rho_c =  1` : perfect agreement
    - `rho_c =  0` : no agreement
    - `rho_c = -1` : perfect disagreement 
    
    Args: 
    - y_true: ground truth
    - y_pred: predicted values
    
    Returns:
    - concordance correlation coefficient (float)
    '''
    
    
    # covariance between y_true and y_pred
    #N = K.int_shape(y_pred)[-1]
    #s_xy = 1.0 / (N - 1.0 + K.epsilon()) * K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    #s_xy = K.mean(K.sum((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred))))
    s_xy = K.mean((y_true - K.mean(y_true)) * (y_pred - K.mean(y_pred)))
    # means
    x_m = K.mean(y_true)
    y_m = K.mean(y_pred)
    # variances
    s_x_sq = K.var(y_true)
    s_y_sq = K.var(y_pred)
    
    # condordance correlation coefficient
    ccc = (2.0*s_xy) / (s_x_sq + s_y_sq + (x_m-y_m)**2+K.epsilon())
    #print(s_xy,s_x_sq,s_y_sq,x_m,y_m)
    return ccc

def CCC_VA(y_true, y_pred):
    return 1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))


In [16]:
import math
def f1_score_max_for_AU_one_class(gt, pred, thresh,type=0):
    gt = gt[:,type]
    pred = pred[:,type]
    P = []
    R = []
    ACC = []
    F1 = []
    for i in thresh:
        new_pred = ((pred >= i) * 1).flatten()
        P.append(precision_score(gt.flatten(), new_pred))
        R.append(recall_score(gt.flatten(), new_pred))
        ACC.append(accuracy_score(gt.flatten(), new_pred))
        F1.append(f1_score(gt.flatten(), new_pred))

    F1_MAX = max(F1)
    if F1_MAX < 0 or math.isnan(F1_MAX):
        F1_MAX = 0
        F1_THRESH = 0
        accuracy = 0
    else:
        idx_thresh = np.argmax(F1)
        F1_THRESH = thresh[idx_thresh]
        accuracy = ACC[idx_thresh]
    return F1,F1_MAX,F1_THRESH,accuracy

def f1_score_max(gt, pred, thresh,c=12):
    F1_s = []
    F1_t = []
    ACC = []
    from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
    for i in range(c):
        F1, F1_MAX, F1_THRESH,acc = f1_score_max_for_AU_one_class(gt,pred,thresh,i)
        F1_s.append(F1_MAX)
        F1_t.append(F1_THRESH)
        ACC.append(acc)
    F1_s=np.array(F1_s)
    F1_t=np.array(F1_t)
    ACC=np.array(ACC)
    return F1_s.mean(),F1_t.mean(),F1_s,F1_t,ACC

## Multi-task challenge 

In [17]:
FEATURES_ONLY,SCORES_ONLY,FEATURES_SCORES=0,1,2
FEATURES_SCORES_AGGREGATION=FEATURES_SCORES
def get_image2all(filename):
    with open(DATA_DIR / filename) as f:
        mtl_lines = f.read().splitlines()
    num_missed=0
    X,y_va,y_expr,y_aus=[],[],[],[]
    masks_va,masks_expr,masks_aus=[],[],[]
    for line in mtl_lines[1:]:
        splitted_line=line.split(',')
        imagename=splitted_line[0]
        valence=float(splitted_line[1])
        arousal=float(splitted_line[2])
        expression=int(splitted_line[3])
        aus=list(map(int,splitted_line[4:]))
        
        if False:
            imagename=imagename[:-4]+'_out.jpg'
            #print(imagename)
        
        mask_VA=(valence>-5 and arousal>-5)
        if not mask_VA:
            valence=arousal=0
            
        mask_expr=(expression>-1)
        if not mask_expr:
            expression=0
            
        mask_aus=min(aus)>=0
        if not mask_aus:
            aus=[0]*len(aus)
        #print(imagename,valence,arousal,expression,aus)
        if mask_VA or mask_expr or mask_aus:
            if imagename in filename2featuresAll:
                if FEATURES_SCORES_AGGREGATION==FEATURES_ONLY:
                    X.append(filename2featuresAll[imagename][0])
                elif FEATURES_SCORES_AGGREGATION==SCORES_ONLY:
                    X.append(filename2featuresAll[imagename][1])
                else:
                    X.append(np.concatenate((filename2featuresAll[imagename][0],filename2featuresAll[imagename][1])))
                y_va.append((valence,arousal))
                masks_va.append(mask_VA)
                
                y_expr.append(expression)
                masks_expr.append(mask_expr)
                
                y_aus.append(aus)
                masks_aus.append(mask_aus)
            else:
                num_missed+=1
        #elif (valence==-5 or arousal==-5) and valence!=arousal:
        #    print(valence,arousal)
    X=np.array(X)
    y_va=np.array(y_va)
    y_expr=np.array(y_expr)
    y_aus=np.array(y_aus)
    masks_va=np.array(masks_va).astype(np.float32)
    masks_expr=np.array(masks_expr).astype(np.float32)
    masks_aus=np.array(masks_aus).astype(np.float32)
    print(X.shape,y_va.shape,y_expr.shape,y_aus.shape,masks_va.shape,num_missed)
    return X,y_va,y_expr,y_aus,masks_va,masks_expr,masks_aus

X_train,y_va_train,y_expr_train,y_aus_train,masks_va_train,masks_expr_train,masks_aus_train=get_image2all('training_set_annotations.txt')
X_val,y_va_val,y_expr_val,y_aus_val,masks_va_val,masks_expr_val,masks_aus_val=get_image2all('validation_set_annotations.txt')
TRAIN_VAL=False

#cropped: (142333, 1288) (142333, 2) (142333,) (142333, 12) (142333,) 0
#(26876, 1288) (26876, 2) (26876,) (26876, 12) (26876,) 0

(142333, 1290) (142333, 2) (142333,) (142333, 12) (142333,) 0
(26876, 1290) (26876, 2) (26876,) (26876, 12) (26876,) 0


In [42]:
if False:
    def softmax(x):
        e_x = np.exp(x - np.max(x,axis=1)[:,np.newaxis])
        return e_x / e_x.sum(axis=1)[:,None]
    X_train=softmax(X_train)
    print(X_train.shape,X_train[:5])
    X_val=softmax(X_val)
    print(X_val.shape,X_val[:5])

In [35]:
if False:
    TRAIN_VAL=True
    X_train=np.concatenate((X_train,X_val))
    y_va_train=np.concatenate((y_va_train,y_va_val))
    y_expr_train=np.concatenate((y_expr_train,y_expr_val))
    y_aus_train=np.concatenate((y_aus_train,y_aus_val))
    masks_va_train=np.concatenate((masks_va_train,masks_va_val))
    masks_expr_train=np.concatenate((masks_expr_train,masks_expr_val))
    masks_aus_train=np.concatenate((masks_aus_train,masks_aus_val))
    print(X_train.shape,y_va_train.shape,y_expr_train.shape,y_aus_train.shape,
          masks_va_train.shape,masks_expr_train.shape,masks_aus_train.shape)

(169209, 1290) (169209, 2) (169209,) (169209, 12) (169209,) (169209,) (169209,)


In [18]:
print(np.unique(y_expr_val[masks_expr_val==1.].astype(int), return_counts=True))

(unique, counts) = np.unique(y_expr_train[masks_expr_train==1.].astype(int), return_counts=True)
num_classes=len(unique)
emo_cw=1/counts
#emo_cw*=counts.mean()
emo_cw/=emo_cw.min()
emo_class_weights = {i:cwi for i,cwi in zip(unique,emo_cw)}
print(counts, emo_class_weights, num_classes, unique)
print(emo_cw)
if False:
    emo_cw=np.array([1,5,5,5,1,1,1])

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([1886,  487,  565, 1254, 3751, 1893, 1003, 4601]))
[23976  4555  3168  3122 18135  7609  5228 24852] {0: 1.0365365365365367, 1: 5.455982436882547, 2: 7.844696969696971, 3: 7.960281870595772, 4: 1.3703887510339123, 5: 3.266132211854383, 6: 4.7536342769701605, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
[1.03653654 5.45598244 7.84469697 7.96028187 1.37038875 3.26613221
 4.75363428 1.        ]


In [19]:
num_labels=y_aus_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
aus_class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_aus_train[masks_aus_train==1., i])
    print(i,neg,pos,np.bincount(y_aus_val[masks_aus_val==1., i]))
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    aus_class_weights[i][0]=weight_for_0
    aus_class_weights[i][1]=weight_for_1
    #aus_class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(aus_class_weights)

12
0 84603 18713 [21345  5531]
1 94571 8745 [24322  2554]
2 82974 20342 [21434  5442]
3 72711 30605 [18058  8818]
4 61979 41337 [13652 13224]
5 65030 38286 [15455 11421]
6 76533 26783 [18557  8319]
7 100856 2460 [26094   782]
8 100336 2980 [26119   757]
9 98313 5003 [26068   808]
10 32732 70584 [ 6449 20427]
11 92560 10756 [23942  2934]
[[ 0.610593    2.7605408 ]
 [ 0.5462351   5.90714694]
 [ 0.62258057  2.53947498]
 [ 0.71045646  1.68789413]
 [ 0.83347585  1.24967946]
 [ 0.79437183  1.34926605]
 [ 0.67497681  1.92876078]
 [ 0.51219561 20.99918699]
 [ 0.5148501  17.33489933]
 [ 0.52544424 10.32540476]
 [ 1.57821093  0.73186558]
 [ 0.55810285  4.80271476]]


In [20]:
def loss_va(y_true, y_pred):
    res=1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))
    #res=K.mean((y_true[:,0] - y_pred[:,0]) * (y_true[:,0] - y_pred[:,0]))+K.mean((y_true[:,1] - y_pred[:,1]) * (y_true[:,1] - y_pred[:,1]))
    #print(res)
    return res

In [21]:
class WeightedExprSCCE(tf.keras.losses.Loss):
    def __init__(self, class_weight, from_logits=False, name='expr_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = tf.keras.losses.Reduction.NONE
        self.unreduced_scce = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        loss=K.mean(loss)
        return loss
loss_expr=WeightedExprSCCE(emo_cw)

In [22]:
if True:
    def get_weighted_loss_aus(weights):
        def weighted_loss(y_true, y_pred):
            y_true=tf.cast(y_true, tf.float32)
            ce=K.binary_crossentropy(y_true, y_pred)
            res=K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce)#, axis=-1)
            #print(res)
            return res
        return weighted_loss
    loss_aus=get_weighted_loss_aus(aus_class_weights)
else:
    pos_weights=np.array([1, 2, 1, 1, 1, 1, 1, 6, 6, 5, 1, 5])
    def loss_aus(y_true, y_pred):
        y_true=tf.cast(y_true, tf.float32)
        ce=K.binary_crossentropy(y_true, y_pred)
        res=K.mean((pos_weights**(y_true))*ce)#, axis=-1)
        #print(res)
        return res

In [23]:
metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] # 

## MTL model 1

In [21]:
num_va_train=masks_va_train.sum()
num_expr_train=masks_expr_train.sum()
num_aus_train=masks_aus_train.sum()
print(num_va_train,num_expr_train,num_aus_train)
print(masks_va_val.sum(),masks_expr_val.sum(),masks_aus_val.sum())

max_num=max(num_va_train,num_expr_train,num_aus_train)
va_weight,expr_weight,aus_weight=max_num/num_va_train,max_num/num_expr_train,max_num/num_aus_train
print(va_weight,expr_weight,aus_weight)

103917.0 90645.0 103316.0
26876.0 15440.0 26876.0
1.0 1.1464174 1.005817


In [64]:
batch_size=256 #128
img = tf.keras.Input(shape=X_train.shape[1:])
mask1 = tf.keras.Input(shape=(1,))
mask2 = tf.keras.Input(shape=(1,))
mask3 = tf.keras.Input(shape=(1,))
x=img
#x=Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
va_out_masked=tf.keras.layers.Multiply(name='va_out')([va_out,mask1])

x=tf.keras.layers.Concatenate(axis=1)([x, va_out])
#x=va_out

aus_out=Dense(12, activation='sigmoid',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
aus_out_masked=tf.keras.layers.Multiply(name='aus_out')([aus_out,mask3])

#x=tf.keras.layers.Concatenate(axis=1)([x, aus_out])
expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
expr_out_masked=tf.keras.layers.Multiply(name='expr_out')([expr_out,mask2])

mtlModel=tf.keras.Model(inputs=[img,mask1,mask2,mask3], outputs=[va_out_masked, expr_out_masked,aus_out_masked])

In [70]:
import copy
if False:
    initial_weights=copy.deepcopy(mtlModel.get_weights())
else:
    mtlModel.set_weights(copy.deepcopy(initial_weights))

In [71]:
mtlModel.compile(optimizer=Adam(lr=1e-3), loss=[loss_va,loss_expr,loss_aus], loss_weights=[va_weight,expr_weight,aus_weight],
                 metrics=[["mean_absolute_error"], ["accuracy"],metrics])
mtlModel.summary()

save_best_model = SaveBestModel('val_loss',False)
mtlModel.fit([X_train,masks_va_train,masks_expr_train,masks_aus_train],
             [y_va_train,y_expr_train,y_aus_train], batch_size=batch_size, epochs=(1 if TRAIN_VAL else 20), 
             verbose=1, callbacks=[save_best_model], 
             validation_data=([X_val,masks_va_val,masks_expr_val,masks_aus_val],[y_va_val,y_expr_val,y_aus_val]))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1290)]       0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            2582        input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 8)            10328       input_5[0][0]                    
____________________________________________________________________________________________

Epoch 19/20
556/556 [==============================] - 6s 11ms/step - loss: 3.4946 - va_out_loss: 0.2684 - expr_out_loss: 2.3042 - aus_out_loss: 0.3561 - va_out_mean_absolute_error: 0.1562 - expr_out_accuracy: 0.8233 - aus_out_aus_out_aus_out_auc: 0.8988 - aus_out_aus_out_aus_out_binary_accuracy: 0.8408 - aus_out_aus_out_aus_out_recall_2: 0.7448 - aus_out_aus_out_aus_out_precision_2: 0.5067 - val_loss: 5.8605 - val_va_out_loss: 0.7890 - val_expr_out_loss: 3.7284 - val_aus_out_loss: 0.5693 - val_va_out_mean_absolute_error: 0.2842 - val_expr_out_accuracy: 0.6358 - val_aus_out_aus_out_aus_out_auc: 0.7754 - val_aus_out_aus_out_aus_out_binary_accuracy: 0.7394 - val_aus_out_aus_out_aus_out_recall_2: 0.7141 - val_aus_out_aus_out_aus_out_precision_2: 0.4872
Epoch 20/20
556/556 [==============================] - 6s 11ms/step - loss: 3.4814 - va_out_loss: 0.2670 - expr_out_loss: 2.2931 - aus_out_loss: 0.3570 - va_out_mean_absolute_error: 0.1556 - expr_out_accuracy: 0.8240 - aus_out_aus_out_aus_o

In [36]:
def print_all():
    y_pred_va,y_pred_expr,y_pred_aus=mtlModel.predict([X_val,masks_va_val,masks_expr_val,masks_aus_val])
    print('\nAV')
    gt_V=y_va_val[masks_va_val==1,0]
    gt_A=y_va_val[masks_va_val==1,1]
    pred_V=y_pred_va[masks_va_val==1,0]
    pred_A=y_pred_va[masks_va_val==1,1]
    ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
    print(gt_V.shape,ccc_V,ccc_A,ccc_VA)
    
    print('\nExpression')
    print(y_expr_val[masks_expr_val==1].shape)
    y_pred=np.argmax(y_pred_expr,axis=1)
    print((y_pred==y_expr_val)[masks_expr_val==1].mean())
    f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
    print(f1_expr)
    print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))
    
    print('\nAUs')
    new_pred = ((y_pred_aus >= 0.5) * 1)
    print(new_pred[masks_aus_val==1,:].shape)
    f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
    print(f1_au)
    print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))
    
    total=ccc_VA+f1_expr+f1_au
    print('\nTotal',ccc_VA,f1_expr,f1_au,total)

In [72]:
print_all()


AV
(26876,) 0.43934089966270284 0.3850961736511451 0.412218536656924

Expression
(15440,)
0.3687176165803109
0.31968436000718103
(0.31968436000718103, 0.3687176165803109, [0.3875251509054326, 0.3013923013923014, 0.5357483317445185, 0.03164556962025317, 0.46005938074367314, 0.33871602993304456, 0.11230856494611458, 0.39007955077211043])

AUs
(26876, 12)
0.46590581560224126


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.49368479684460276, 0.5750000000000001, array([0.56099218, 0.44302449, 0.56837571, 0.56656315, 0.72636587,
       0.70537923, 0.66032561, 0.13768524, 0.1047619 , 0.21239954,
       0.87793932, 0.3604053 ]), array([0.6, 0.7, 0.6, 0.5, 0.4, 0.5, 0.5, 0.6, 0.7, 0.8, 0.3, 0.7]), array([0.80770948, 0.88324155, 0.80610954, 0.6884209 , 0.68972317,
       0.73853996, 0.78341271, 0.8289552 , 0.96502456, 0.94895074,
       0.80164459, 0.84733591]))

Total 0.412218536656924 0.31968436000718103 0.46590581560224126 1.1978087122663463


In [73]:
print('Best weights:')
mtlModel.set_weights(best_model_weights)
print_all()

Best weights:

AV
(26876,) 0.446480673138777 0.3701610243422664 0.40832084874052166

Expression
(15440,)
0.38335492227979273
0.32960438325217256
(0.32960438325217256, 0.38335492227979273, [0.3923031612016493, 0.3071230342275671, 0.5136921624173748, 0.053571428571428575, 0.4546750285062714, 0.4235209235209235, 0.08527648234510327, 0.4066728452270621])

AUs
(26876, 12)
0.4648836547544832


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.49390624765786967, 0.5666666666666668, array([0.55765854, 0.43717001, 0.56655927, 0.56776138, 0.72728473,
       0.70859504, 0.66182415, 0.13815926, 0.10613944, 0.21597957,
       0.87757485, 0.36216875]), array([0.7, 0.7, 0.6, 0.5, 0.4, 0.5, 0.5, 0.6, 0.7, 0.7, 0.2, 0.7]), array([0.8312993 , 0.88100908, 0.80979312, 0.69050454, 0.69255097,
       0.73760976, 0.78519869, 0.84495461, 0.9680384 , 0.92004018,
       0.7932356 , 0.84417324]))

Total 0.40832084874052166 0.32960438325217256 0.4648836547544832 1.2028088867471773


In [65]:
if False:
    #mtlModel.save_weights('mtl_enet2_8.h5') #0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802
    #mtlModel.save_weights('mtl_enet2_8.h5') #0.38941268091453485 0.2966496393771772 0.46193077632133167 1.1479930966130436
    #mtlModel.save_weights('mtl_enet0_vgaf.h5') #0.4101470046160306 0.30374060068461467 0.4453343451586555 1.1592219504593009
    #mtlModel.save_weights('mtl_enet0_mtl_scores.h5') #0.41964194296471674 0.3108849599582243 0.4495678236137662 1.1800947265367072
    #mtlModel.save_weights('mtl_enet0_mtl_features.h5') #0.4077160039302934 0.3304590574311655 0.46342233378866404 1.2015973951501229
    mtlModel.save_weights('mtl_enet0_mtl_features_new.h5') #0.4205420369528933 0.32631765971330484 0.4666244179960605 1.2134841146622586
    #0.41443352670757555 0.33392390239905834 0.4631641154858293 1.2115215445924632
else:
    #mtlModel.load_weights('mtl_enet2_8_orig.h5')
    mtlModel.load_weights('mtl_enet0_vgaf.h5')
    #mtlModel.load_weights('mtl_enet0_mtl_features_new.h5')
    print_all()


AV
(26876,) 0.441977966779619 0.3783160424524422 0.4101470046160306

Expression
(15440,)
0.3645077720207254
0.30374060068461467
(0.30374060068461467, 0.3645077720207254, [0.3225108225108225, 0.306320907617504, 0.36872812135355904, 0.0, 0.48519040902679833, 0.3380281690140845, 0.2184441009788769, 0.39070227497527205])

AUs
(26876, 12)
0.4453343451586555


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4762028954542939, 0.5333333333333333, array([0.53098316, 0.42112483, 0.54961963, 0.55606101, 0.71629462,
       0.69021277, 0.65632557, 0.11511433, 0.07011592, 0.17885679,
       0.87514832, 0.35457781]), array([0.6, 0.7, 0.6, 0.4, 0.4, 0.5, 0.5, 0.5, 0.6, 0.7, 0.2, 0.7]), array([0.7772362 , 0.87438607, 0.80395148, 0.62963239, 0.67796547,
       0.72912636, 0.78318946, 0.70482959, 0.87762316, 0.90058044,
       0.78858461, 0.85154041]))

Total 0.4101470046160306 0.30374060068461467 0.4453343451586555 1.1592219504593009


mobilenet
output layers only: 0.3575652288180158 0.274931498498287 0.4667221169978419 1.0992188443141446
0.35813183800284104 0.2823732083688107 0.47113929424277273 1.1116443406144245
dense layer: 0.3212823862697798 0.25187087483194115 0.46361689956738755 1.0367701606691084

enet0_7
output layers only:0.37147839426933726 0.2862848390733215 0.4561286864943848 1.1138919198370436
dense layer: 0.37387452943730165 0.2522474413566924 0.4641164043127432 1.0902383751067373
not best 0.3944477107433102 0.23835206524054334 0.4603934421520726 1.093193218135926

enet_b0_8_best_afew
features+emotions
output layers only:0.4026537613423008 0.248547670046962 0.45890535417538064 1.1101067855646434


enet_b0_8_best_vgaf
features+emotions
output layers only:0.38615587195723955 0.282608775233668 0.4545389225496568 1.1233035697405644
dense: 0.3808455198959431 0.27195008480301336 0.46871028488210437 1.1215058895810608

SR:
0.37119770014719833 0.2455657651558003 0.45512883392676473 1.0718922992297633
aligned:
0.39612867971292187 0.29007220939613076 0.4618770485417234 1.148077937650776

scores, new model (concat VA): 0.4101470046160306 0.30374060068461467 0.4453343451586555 1.1592219504593009

mtl_enet0_mtl: 
scores only: 0.41964194296471674 0.3108849599582243 0.4495678236137662 1.1800947265367072
features only: 0.4148349771535716 0.30801118050190446 0.4667824146670865 1.1896285723225626
scores+features: 
new network: 0.41443352670757555 0.33392390239905834 0.4631641154858293 1.2115215445924632
previous: 0.4206581824810957 0.32312430364131917 0.4612318367279269 1.2050143228503418

SR:scores 0.397964255040454 0.2888308244878144 0.4556445368596133 1.1424396163878818


enet2_8
output layers only: 0.38307364079552575 0.28723802667892157 0.4593295818373552 1.1296412493118027
dense layer: 0.3966562712594687 0.281604125663917 0.45615746671167773 1.1344178636350635

features+emotions
0.3837151018856935 0.3022424974391059 0.46080879636948063 1.1467663956942802
+dense: 0.3779975030622512 0.2987745729052591 0.45822890901489305 1.1350009849824034


enet2_7
features
output layers only: 0.39352969999919685 0.2741800561672634 0.4611839520090892 1.1288937081755495
dense layer: 0.3678038228570031 0.2800059725435074 0.4564833238081867 1.104293119208697

features+emotions
output layers only: 0.38307364079552575 0.28723802667892157 0.4593295818373552 1.1296412493118027
dense layer:  0.3919721221529454 0.2821609899822945 0.460412089791669 1.1345452019269089

## MTL model 2

In [24]:
X_expr_only_train,y_expr_only_train=X_train[masks_expr_train==1],y_expr_train[masks_expr_train==1]
X_expr_only_val,y_expr_only_val=X_val[masks_expr_val==1],y_expr_val[masks_expr_val==1]
print(X_expr_only_train.shape,y_expr_only_train.shape,X_expr_only_val.shape,y_expr_only_val.shape)

X_va_only_train,y_va_only_train=X_train[masks_va_train==1],y_va_train[masks_va_train==1]
X_va_only_val,y_va_only_val=X_val[masks_va_val==1],y_va_val[masks_va_val==1]
print(X_va_only_train.shape,y_va_only_train.shape,X_va_only_val.shape,y_va_only_val.shape)

X_au_only_train,y_au_only_train=X_train[masks_aus_train==1],y_aus_train[masks_aus_train==1]
X_au_only_val,y_au_only_val=X_val[masks_aus_val==1],y_aus_val[masks_aus_val==1]
print(X_au_only_train.shape,y_au_only_train.shape,X_au_only_val.shape,y_au_only_val.shape)

(90645, 1290) (90645,) (15440, 1290) (15440,)
(103917, 1290) (103917, 2) (26876, 1290) (26876, 2)
(103316, 1290) (103316, 12) (26876, 1290) (26876, 12)


In [25]:
class Slice(tf.keras.layers.Layer):
    def __init__(self, begin, size,**kwargs):
        super(Slice, self).__init__(**kwargs)
        self.begin = begin
        self.size = size
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'begin': self.begin,
            'size': self.size,
        })
        return config
    def call(self, inputs):
        #return tf.slice(inputs, self.begin, self.size)
        return inputs[:,self.begin:self.begin+self.size]

In [26]:
batch_size=512
img = tf.keras.Input(shape=X_train.shape[1:])
x=img
#x=Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
if True:
    expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    s_l=Slice(0,1280)(x)
    expr_out=Dense(8, activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(s_l)
if False:
    va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    s_l=Slice(1280,10)(x)
    va_out = Dense(2, activation='tanh',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(s_l)
    
if False:
    aus_out=Dense(12, activation='sigmoid',kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size))(x)
else:
    au_fc=Dense(128,activation='relu')(x)
    aus_out=Dense(12, activation='sigmoid')(au_fc)

mtl_model_new=tf.keras.Model(inputs=img, outputs=[expr_out, va_out, aus_out])

In [27]:
import copy
if True:
    initial_weights=copy.deepcopy(mtl_model_new.get_weights())
else:
    mtl_model_new.set_weights(copy.deepcopy(initial_weights))

### Expression

In [28]:
model_expr=tf.keras.Model(inputs=img, outputs=expr_out)
model_expr.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_expr.summary()

save_best_model = SaveBestModel('val_accuracy',True)
model_expr.fit(X_expr_only_train,y_expr_only_train, batch_size=batch_size, epochs=(1 if TRAIN_VAL else 10), verbose=1, 
             callbacks=[save_best_model], validation_data=(X_expr_only_val,y_expr_only_val),class_weight=emo_class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1290)]            0         
_________________________________________________________________
dense (Dense)                (None, 8)                 10328     
Total params: 10,328
Trainable params: 10,328
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
178/178 [==============================] - 1s 5ms/step - loss: 3.3695 - accuracy: 0.4528 - val_loss: 1.8048 - val_accuracy: 0.3453
Epoch 2/10
178/178 [==============================] - 1s 5ms/step - loss: 2.1699 - accuracy: 0.6471 - val_loss: 1.8506 - val_accuracy: 0.3508
Epoch 3/10
178/178 [==============================] - 1s 5ms/step - loss: 1.9914 - accuracy: 0.6855 - val_loss: 1.9130 - val_accuracy: 0.3360
Epoch 4/10
178/178 [==============================] - 1s 5ms/step - loss: 1.9104 - accuracy

In [29]:
def print_expr():
    y_val_preds,_,_=mtl_model_new.predict(X_val)
    y_pred=np.argmax(y_val_preds,axis=1)[masks_expr_val==1]
    print('Acc:',(y_pred==y_expr_only_val).mean(), 'F1:',f1_score(y_true=y_expr_only_val,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y_expr_only_val,y_pred))

In [30]:
if not TRAIN_VAL:
    print_expr()
    print('Best weights:')
    model_expr.set_weights(best_model_weights)
print_expr()

Acc: 0.331800518134715 F1: 0.31392339895916155
(0.31392339895916155, 0.331800518134715, [0.34149926506614403, 0.21656686626746505, 0.3997728563316298, 0.22598870056497175, 0.47736865085140445, 0.39370654957921697, 0.16640986132511557, 0.2900744416873449])
Best weights:
Acc: 0.3508419689119171 F1: 0.3362736789002177
(0.3362736789002177, 0.3508419689119171, [0.34167468719923, 0.2123015873015873, 0.41098901098901097, 0.27903469079939663, 0.4929203539823009, 0.46383701188455, 0.1916354556803995, 0.2977966333652662])


features only
Acc: 0.35582901554404145 F1: 0.33853939646012643
(0.33853939646012643, 0.35582901554404145, [0.3362332471196803, 0.20376629647513278, 0.4379915305505142, 0.2291891891891892, 0.5075120192307693, 0.4421906693711967, 0.22203048605119355, 0.32940173369333525])

scores only
Best weights:
Acc: 0.280699481865285 F1: 0.2605997985451292
(0.2605997985451292, 0.280699481865285, [0.29420360448124694, 0.18459495351925634, 0.24043715846994537, 0.1410153102336825, 0.46742292030250154, 0.37460913070669166, 0.2737528202557032, 0.10876249039200615])

scores+features
Acc: 0.3480569948186529 F1: 0.3334845842529065
(0.3334845842529065, 0.3480569948186529, [0.3404950957496497, 0.21566632756866735, 0.41048522030117124, 0.28926553672316385, 0.47647409172126265, 0.4472135687088958, 0.18200134318334452, 0.3062754900670965])


### Valence-arousal 

In [31]:
model_va=tf.keras.Model(inputs=img, outputs=va_out)
model_va.compile(optimizer=Adam(lr=1e-3), loss=CCC_VA, metrics=['mae','mse'])
model_va.summary()

save_best_model = SaveBestModel('val_loss',False)
model_va.fit(X_va_only_train,y_va_only_train, batch_size=batch_size, epochs=(8 if TRAIN_VAL else 20), verbose=1, callbacks=[save_best_model], 
          validation_data=(X_va_only_val,y_va_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1290)]            0         
_________________________________________________________________
slice (Slice)                (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
203/203 [==============================] - 2s 6ms/step - loss: 1.1153 - mae: 0.7420 - mse: 0.7599 - val_loss: 0.9710 - val_mae: 0.6732 - val_mse: 0.6497
Epoch 2/20
203/203 [==============================] - 1s 5ms/step - loss: 0.9301 - mae: 0.6797 - mse: 0.6436 - val_loss: 0.9086 - val_mae: 0.6148 - val_mse: 0.5445
Epoch 3/20
203/203 [==============================] - 1s 6ms/step

In [32]:
def print_ccc():
    _,y_val_preds,_=mtl_model_new.predict(X_val)
    pred_V=y_val_preds[masks_va_val==1,0]
    pred_A=y_val_preds[masks_va_val==1,1]
    gt_V=y_va_only_val[:,0]
    gt_A=y_va_only_val[:,1]
    print(metric_for_VA(gt_V,gt_A,pred_V,pred_A))
    print(CCC_numpy(gt_V,pred_V),CCC_numpy(gt_A,pred_A))

In [33]:
if not TRAIN_VAL:
    print_ccc()
    print('Best weights:')
    model_va.set_weights(best_model_weights)
print_ccc()

(0.4640283582877984, 0.4195844716777797, 0.44180641498278905)
0.4640456021571796 0.41960007897731355
Best weights:
(0.462840615766619, 0.4192827615479205, 0.44106168865726975)
0.4628578275639917 0.41929834856504755


features only
(0.44702991590919305, 0.3999768085100121, 0.4235033622096026)
0.4470465505604636 0.39999170544933943

scores only
(0.4678666794930614, 0.41790110240886674, 0.44288389095096403)
0.4678840728855078 0.4179166374299864

scores+features
(0.4471655368461229, 0.4057897961501993, 0.4264776664981611)
0.4471821561384188 0.4058049134330271
Best weights:
(0.4290930580201086, 0.40585204778442285, 0.4174725529022657)
0.42910903221993124 0.4058671776723511

### Action Units 

In [34]:
model_au=tf.keras.Model(inputs=img, outputs=aus_out)
model_au.compile(optimizer=Adam(lr=1e-3), loss=loss_aus, metrics=metrics)
model_au.summary()

save_best_model = SaveBestModel('val_loss',False)
model_au.fit(X_au_only_train,y_au_only_train, batch_size=batch_size, epochs=(3 if TRAIN_VAL else 10), verbose=1, callbacks=[save_best_model], 
          validation_data=(X_au_only_val,y_au_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1290)]            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               165248    
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1548      
Total params: 166,796
Trainable params: 166,796
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
202/202 [==============================] - 2s 8ms/step - loss: 0.4855 - auc: 0.8428 - binary_accuracy: 0.7752 - recall: 0.7436 - precision: 0.5025 - val_loss: 0.5610 - val_auc: 0.8030 - val_binary_accuracy: 0.7618 - val_recall: 0.7206 - val_precision: 0.5187
Epoch 2/10
202/202 [==============================] - 2s 8ms/step - loss: 0.3505 - auc: 0.9229 - binary_accuracy:

In [35]:
def print_au():
    _,_,y_val_preds=mtl_model_new.predict(X_val)
    y_val_preds=y_val_preds[masks_va_val==1,:]
    new_pred = ((y_val_preds >= 0.5) * 1)
    print(np.mean([f1_score(y_true=y_au_only_val[:,i],y_pred=new_pred[:,i]) for i in range(y_val_preds.shape[1])]))
    print(f1_score_max(y_au_only_val,y_val_preds,thresh=np.arange(0.1,1,0.1)))

In [36]:
if not TRAIN_VAL:
    print_au()
    print('Best weights:')
    model_au.set_weights(best_model_weights)
print_au()

0.493053875926913
(0.512953298361032, 0.55, array([0.57497415, 0.42757318, 0.58180434, 0.59515843, 0.73442735,
       0.70933747, 0.65871455, 0.2402863 , 0.2105919 , 0.17330046,
       0.87640603, 0.37286542]), array([0.6, 0.8, 0.3, 0.3, 0.4, 0.5, 0.6, 0.8, 0.9, 0.4, 0.2, 0.8]), array([0.81649055, 0.8872228 , 0.79561691, 0.69385325, 0.71096889,
       0.73522846, 0.79985861, 0.94470903, 0.95285757, 0.91267302,
       0.79762613, 0.8510567 ]))
Best weights:
0.4840685140661731
(0.5209951567187995, 0.6166666666666666, array([0.58488206, 0.46218487, 0.58164797, 0.61660118, 0.73464316,
       0.71510358, 0.67256086, 0.23215686, 0.17610619, 0.22811918,
       0.88039045, 0.36754551]), array([0.7, 0.8, 0.6, 0.4, 0.4, 0.4, 0.6, 0.9, 0.8, 0.8, 0.2, 0.8]), array([0.82713201, 0.89045989, 0.82374609, 0.70955499, 0.70602024,
       0.71855931, 0.80432356, 0.96357345, 0.93071886, 0.93830927,
       0.80577467, 0.85005209]))


features only
0.4877955052186667
(0.5189570784821854, 0.5833333333333334, array([0.58116199, 0.45216079, 0.57108141, 0.61143747, 0.73423989,
       0.71524042, 0.67407235, 0.24712644, 0.18706157, 0.21825599,
       0.87847458, 0.35717205]), array([0.6, 0.8, 0.5, 0.4, 0.4, 0.5, 0.5, 0.8, 0.8, 0.8, 0.2, 0.7]), array([0.81518827, 0.89198541, 0.80298408, 0.71204792, 0.70840155,
       0.73999107, 0.79148683, 0.95125763, 0.92238428, 0.94296026,
       0.80056556, 0.82441584]))


scores only
0.45390063440662276
(0.48795630442917454, 0.6083333333333335, array([0.54249909, 0.41938422, 0.58782223, 0.55747069, 0.71403767,
       0.69672571, 0.66193509, 0.12175394, 0.09201389, 0.23825503,
       0.87423195, 0.34934614]), array([0.7, 0.8, 0.7, 0.4, 0.4, 0.5, 0.5, 0.8, 0.7, 0.9, 0.2, 0.7]), array([0.81254651, 0.88141837, 0.83401548, 0.67137967, 0.68135139,
       0.72670784, 0.79264027, 0.92324007, 0.92216104, 0.93243042,
       0.79360768, 0.81857419]))
Best weights:
0.4537206686412025
(0.4870261851387019, 0.5916666666666667, array([0.5471062 , 0.42240552, 0.5691379 , 0.55744759, 0.72267255,
       0.69784564, 0.66115412, 0.11682172, 0.08278146, 0.24589348,
       0.87535587, 0.34569218]), array([0.7, 0.8, 0.7, 0.4, 0.4, 0.5, 0.5, 0.6, 0.7, 0.9, 0.2, 0.7]), array([0.82006251, 0.88796696, 0.83003423, 0.64027385, 0.69575086,
       0.73124721, 0.78501265, 0.78452895, 0.91754725, 0.94363   ,
       0.79148683, 0.81745796]))
       

scores+features
0.4858337764396418
(0.5187352880665755, 0.6166666666666667, array([0.58193373, 0.45640637, 0.56947413, 0.59397876, 0.73822173,
       0.71687038, 0.66788597, 0.25583982, 0.1667372 , 0.2451184 ,
       0.88012497, 0.352232  ]), array([0.7, 0.8, 0.6, 0.4, 0.5, 0.5, 0.6, 0.8, 0.7, 0.8, 0.2, 0.8]), array([0.82865754, 0.89328769, 0.81265813, 0.70695044, 0.7213127 ,
       0.73798184, 0.8040631 , 0.95021581, 0.92673761, 0.93239321,
       0.80298408, 0.85206132]))

### Combine all 

In [37]:
if False:
    #mtl_model_new.save_weights('mtl_enet0_separate.h5') #0.4416453346692747 0.338425880038155 0.4892950104555161 1.2693662251629458
    mtl_model_new.save_weights('mtl_enet0_slice.h5') #0.4470512345513264 0.33507035885799447 0.4943377041019861 1.276459297511307
else:
    mtl_model_new.load_weights('../../../emotions-multimodal/faces/ABAW/abaw4/'+'mtl_enet0_slice.h5')

In [38]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_val)
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nExpression')
print(y_expr_val[masks_expr_val==1].shape)
y_pred=np.argmax(y_pred_expr,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_per_au=[f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])]
f1_au=np.mean(f1_per_au)
print(f1_au,f1_per_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
(26876,) 0.4749176554355453 0.41918481366710747 0.4470512345513264

Expression
(15440,)
0.35174870466321245
0.33507035885799447
(0.33507035885799447, 0.35174870466321245, [0.3309143686502177, 0.21268097853220172, 0.4306987399770905, 0.2674139311449159, 0.4919247295895688, 0.45335129673290675, 0.17721518987341772, 0.31636363636363635])

AUs
(26876, 12)
0.4943377041019861 [0.5768960399906272, 0.4068865816795899, 0.5824795488841237, 0.5856791309045075, 0.7315399891677198, 0.7165887948815086, 0.6772090777402222, 0.1664488017429194, 0.12901334115232369, 0.17010197439791713, 0.8462372811321728, 0.34297188755020075]
(0.5217870390537378, 0.5583333333333335, array([0.58350767, 0.46639283, 0.58247955, 0.60487214, 0.73661289,
       0.71948443, 0.67720908, 0.25145518, 0.18346774, 0.2108389 ,
       0.88044481, 0.36467925]), array([0.6, 0.7, 0.5, 0.3, 0.4, 0.4, 0.5, 0.9, 0.8, 0.7, 0.2, 0.7]), array([0.82222057, 0.89365977, 0.80439798, 0.68436523, 0.70296919,
       0.72953565, 0.80101206, 0.95

features only
Total 0.4235033622096026 0.33853939646012643 0.4877955052186667 1.2498382638883958

scores only
Total 0.44288389095096403 0.2605997985451292 0.4537206686412025 1.1572043581372957

scores+features
0.4174725529022657 0.3334845842529065 0.4858337764396418 1.236790913594814

## OpenFace 

In [34]:
data_dir=DATA_DIR / 'cropped_aligned'
#outdir=DATA_DIR / 'openface_cropped_aligned'
outdir=DATA_DIR / 'openface_aus_cropped_aligned'
test_data_dir=DATA_DIR / 'mtl_test_data/cropped_aligned'
test_outdir=DATA_DIR / 'mtl_test_data/openface_aus_cropped_aligned'

if False:#Test
    data_dir=test_data_dir
    outdir=test_outdir
print(data_dir,outdir)

/home/HDD6TB/datasets/emotions/ABAW/eccv_4/cropped_aligned /home/HDD6TB/datasets/emotions/ABAW/eccv_4/openface_aus_cropped_aligned


In [21]:
if not os.path.exists(outdir):
    os.makedirs(outdir)
cur_dir=os.getcwd()
os.chdir(data_dir)
#command='/home/avsavchenko/src/distr/OpenFace/build/bin/FeatureExtraction -pose -aus -gaze -out_dir '+str(outdir)
command='/home/avsavchenko/src/distr/OpenFace/build/bin/FeatureExtraction -aus -out_dir '+str(outdir)
for filename in os.listdir('.'):
    if os.path.isdir(filename):
        command+=' -fdir '+filename

print(command)
os.system(command=command)

os.chdir(cur_dir)
print(os.getcwd())

/home/avsavchenko/src/distr/OpenFace/build/bin/FeatureExtraction -aus -out_dir /home/HDD6TB/datasets/emotions/ABAW/eccv_4/mtl_test_data/openface_aus_cropped_aligned -fdir video32 -fdir video17 -fdir 197 -fdir video21 -fdir video5_right -fdir 171 -fdir 199 -fdir video45_4 -fdir 134-30-1280x720 -fdir 241 -fdir 204 -fdir video12 -fdir video53 -fdir video26 -fdir video5_left -fdir video33 -fdir video13 -fdir 286 -fdir 322 -fdir video22 -fdir video39 -fdir video35 -fdir video20 -fdir 274 -fdir 14-30-1920x1080 -fdir 130-25-1280x720_left -fdir 219 -fdir 270 -fdir video51 -fdir video45_1 -fdir video45_2 -fdir 234 -fdir video37 -fdir 304 -fdir video38 -fdir video29_right -fdir video46 -fdir video30 -fdir 49-30-1280x720_right -fdir 212 -fdir 40-30-1280x720 -fdir 265 -fdir video11 -fdir 267 -fdir video19 -fdir video23 -fdir video42 -fdir video25 -fdir video16 -fdir video49_right -fdir video9 -fdir video40 -fdir 239 -fdir 194 -fdir video44 -fdir 281 -fdir 313 -fdir 284 -fdir 130-25-1280x720_right 

In [35]:
def get_openface_features(filename,success_only=True):
    with open(DATA_DIR / filename) as f:
        mtl_lines = f.read().splitlines()
    num_missed=num_fails=0
    X,y_va,y_expr,y_aus=[],[],[],[]
    masks_va,masks_expr,masks_aus=[],[],[]
    mask_success=[]
    
    prev_videoname=''
    video_frames=None
    
    for line in mtl_lines[1:]:
        splitted_line=line.split(',')
        imagename=splitted_line[0]
        valence=float(splitted_line[1])
        arousal=float(splitted_line[2])
        expression=int(splitted_line[3])
        aus=list(map(int,splitted_line[4:]))
        
        mask_VA=(valence>-5 and arousal>-5)
        if not mask_VA:
            valence=arousal=0
            
        mask_expr=(expression>-1)
        if not mask_expr:
            expression=0
            
        mask_aus=min(aus)>=0
        if not mask_aus:
            aus=[0]*len(aus)
        #print(imagename,valence,arousal,expression,aus)
        if mask_VA or mask_expr or mask_aus:
            videoname,filename=imagename.split('/')
            if videoname!=prev_videoname:
                fpath=outdir / (videoname+'.csv')
                if os.path.exists(fpath):
                    df = pd.read_csv(fpath)
                    success=(df.success==1).to_numpy()
                    df = df.drop(columns=['frame', 'face_id','timestamp', 'confidence', 'success'])
                    df=df.drop(columns=[c for c in df.columns if c.startswith('eye')])
                    df=df.drop(columns=[c for c in df.columns if c.startswith('gaze')])
                    #df=df.drop(columns=[c for c in df.columns if c.startswith('AU')])
                    df=df.drop(columns=[c for c in df.columns if c.startswith('pose')])
                    cur_features=df.to_numpy()
                    
                    video_frames={f:i for i,f in enumerate(sorted(os.listdir(data_dir / videoname)))}
                    #print(videoname,cur_features.shape,len(video_frames))
                    prev_videoname=videoname
                
            if video_frames is not None and filename in video_frames:
                ind=video_frames[filename]
                if not success_only or success[ind]==1:
                    mask_success.append(success[ind])
                    X.append(cur_features[ind])
                    y_va.append((valence,arousal))
                    masks_va.append(mask_VA)

                    y_expr.append(expression)
                    masks_expr.append(mask_expr)

                    y_aus.append(aus)
                    masks_aus.append(mask_aus)
                if not success[ind]:
                    num_fails+=1
            else:
                num_missed+=1
        #elif (valence==-5 or arousal==-5) and valence!=arousal:
        #    print(valence,arousal)
    X=np.array(X)
    y_va=np.array(y_va)
    y_expr=np.array(y_expr)
    y_aus=np.array(y_aus)
    masks_va=np.array(masks_va).astype(np.float32)
    masks_expr=np.array(masks_expr).astype(np.float32)
    masks_aus=np.array(masks_aus).astype(np.float32)
    mask_success=np.array(mask_success)
    print(X.shape,y_va.shape,y_expr.shape,y_aus.shape,masks_va.shape,mask_success.shape,num_missed,mask_success.sum(),num_fails)
    return X,y_va,y_expr,y_aus,masks_va,masks_expr,masks_aus,mask_success

X_of_train,y_of_va_train,y_of_expr_train,y_of_aus_train,masks_of_va_train,masks_of_expr_train,masks_of_aus_train,_=get_openface_features('training_set_annotations.txt',success_only=True)
X_of_val_all,y_of_va_val_all,y_of_expr_val_all,y_of_aus_val_all,masks_of_va_val_all,masks_of_expr_val_all,masks_of_aus_val_all,masks_val_success=get_openface_features('validation_set_annotations.txt',success_only=False)
X_of_val,y_of_va_val,y_of_expr_val,y_of_aus_val,masks_of_va_val,masks_of_expr_val,masks_of_aus_val=X_of_val_all[masks_val_success],y_of_va_val_all[masks_val_success],y_of_expr_val_all[masks_val_success],y_of_aus_val_all[masks_val_success],masks_of_va_val_all[masks_val_success],masks_of_expr_val_all[masks_val_success],masks_of_aus_val_all[masks_val_success]
TRAIN_VAL=False

(133568, 35) (133568, 2) (133568,) (133568, 12) (133568,) (133568,) 0 133568 8765
(26876, 35) (26876, 2) (26876,) (26876, 12) (26876,) (26876,) 0 25796 1080


In [36]:
X_of_expr_only_train,y_of_expr_only_train=X_of_train[masks_of_expr_train==1],y_of_expr_train[masks_of_expr_train==1]
X_of_expr_only_val,y_of_expr_only_val=X_of_val[masks_of_expr_val==1],y_of_expr_val[masks_of_expr_val==1]
print(X_of_expr_only_train.shape,y_of_expr_only_train.shape,X_of_expr_only_val.shape,y_of_expr_only_val.shape)

X_of_va_only_train,y_of_va_only_train=X_of_train[masks_of_va_train==1],y_of_va_train[masks_of_va_train==1]
X_of_va_only_val,y_of_va_only_val=X_of_val[masks_of_va_val==1],y_of_va_val[masks_of_va_val==1]
print(X_of_va_only_train.shape,y_of_va_only_train.shape,X_of_va_only_val.shape,y_of_va_only_val.shape)

X_of_au_only_train,y_of_au_only_train=X_of_train[masks_of_aus_train==1],y_of_aus_train[masks_of_aus_train==1]
X_of_au_only_val,y_of_au_only_val=X_of_val[masks_of_aus_val==1],y_of_aus_val[masks_of_aus_val==1]
print(X_of_au_only_train.shape,y_of_au_only_train.shape,X_of_au_only_val.shape,y_of_au_only_val.shape)

(84573, 35) (84573,) (14717, 35) (14717,)
(97977, 35) (97977, 2) (25796, 35) (25796, 2)
(97376, 35) (97376, 12) (25796, 35) (25796, 12)


In [37]:
(unique, counts) = np.unique(y_of_expr_only_train.astype(int), return_counts=True)
num_classes=len(unique)
emo_cw=1/counts
#emo_cw*=counts.mean()
emo_cw/=emo_cw.min()
emo_class_weights = {i:cwi for i,cwi in zip(unique,emo_cw)}
print(counts, emo_class_weights, num_classes, unique)
print(emo_cw)


num_labels=y_of_aus_train.shape[1]
print(num_labels)
from sklearn.utils.class_weight import compute_class_weight
aus_class_weights = np.empty([num_labels, 2])
for i in range(num_labels):
    neg, pos = np.bincount(y_of_au_only_train[:, i])
    print(i,neg,pos,np.bincount(y_of_au_only_val[:, i]))
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    aus_class_weights[i][0]=weight_for_0
    aus_class_weights[i][1]=weight_for_1
    #aus_class_weights[i] = compute_class_weight('balanced', [0,1], y_train[:, i])
print(aus_class_weights) 

[22485  4335  3130  2720 16643  7202  4626 23432] {0: 1.0421169668668, 1: 5.405305651672434, 2: 7.486261980830671, 3: 8.614705882352942, 4: 1.4079192453283662, 5: 3.253540683143571, 6: 5.065283182014699, 7: 1.0} 8 [0 1 2 3 4 5 6 7]
[1.04211697 5.40530565 7.48626198 8.61470588 1.40791925 3.25354068
 5.06528318 1.        ]
12
0 79848 17528 [20491  5305]
1 89126 8250 [23289  2507]
2 78020 19356 [20659  5137]
3 68420 28956 [17371  8425]
4 58373 39003 [12913 12883]
5 61589 35787 [14731 11065]
6 71971 25405 [17663  8133]
7 94982 2394 [25017   779]
8 94445 2931 [25045   751]
9 92583 4793 [25009   787]
10 30861 66515 [ 6066 19730]
11 87156 10220 [22915  2881]
[[ 0.60975854  2.77772707]
 [ 0.54628279  5.90157576]
 [ 0.62404512  2.51539574]
 [ 0.71160479  1.68144771]
 [ 0.83408425  1.24831423]
 [ 0.79053078  1.36049403]
 [ 0.6764947   1.91647314]
 [ 0.51260239 20.33751044]
 [ 0.51551697 16.61139543]
 [ 0.52588488 10.1581473 ]
 [ 1.57765465  0.73198527]
 [ 0.5586305   4.76399217]]


In [38]:
def loss_va(y_true, y_pred):
    res=1-0.5*(CCC(y_true[:,0], y_pred[:,0])+CCC(y_true[:,1], y_pred[:,1]))
    #res=K.mean((y_true[:,0] - y_pred[:,0]) * (y_true[:,0] - y_pred[:,0]))+K.mean((y_true[:,1] - y_pred[:,1]) * (y_true[:,1] - y_pred[:,1]))
    #print(res)
    return res

class WeightedExprSCCE(tf.keras.losses.Loss):
    def __init__(self, class_weight, from_logits=False, name='expr_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = tf.keras.losses.Reduction.NONE
        self.unreduced_scce = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        loss=K.mean(loss)
        return loss
loss_expr=WeightedExprSCCE(emo_cw)



def get_weighted_loss_aus(weights):
    def weighted_loss(y_true, y_pred):
        y_true=tf.cast(y_true, tf.float32)
        ce=K.binary_crossentropy(y_true, y_pred)
        res=K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*ce)#, axis=-1)
        #print(res)
        return res
    return weighted_loss
loss_aus=get_weighted_loss_aus(aus_class_weights)

metrics=[tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()] #

## Expr 

In [45]:
import copy
batch_size=256 #128
model_expr=Sequential()
if False:
    model_expr.add(Dense(num_classes, input_shape=X_of_train.shape[1:],activation='softmax',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    model_expr.add(Dense(128, input_shape=X_of_train.shape[1:],activation='relu')) #256
    model_expr.add(Dense(num_classes,activation='softmax'))
    
initial_weights=copy.deepcopy(model_expr.get_weights())

In [49]:
model_expr.set_weights(copy.deepcopy(initial_weights))
model_expr.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_expr.summary()

save_best_model = SaveBestModel('val_accuracy',True)
model_expr.fit(X_of_expr_only_train,y_of_expr_only_train, batch_size=batch_size, epochs=10, verbose=1, 
             callbacks=[save_best_model], validation_data=(X_of_expr_only_val,y_of_expr_only_val),class_weight=emo_class_weights)
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               4608      
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 1032      
Total params: 5,640
Trainable params: 5,640
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
331/331 [==============================] - 1s 4ms/step - loss: 4.1472 - accuracy: 0.2664 - val_loss: 1.9916 - val_accuracy: 0.2615
Epoch 2/10
331/331 [==============================] - 1s 4ms/step - loss: 3.4199 - accuracy: 0.3742 - val_loss: 1.9665 - val_accuracy: 0.2772
Epoch 3/10
331/331 [==============================] - 1s 3ms/step - loss: 3.1885 - accuracy: 0.4117 - val_loss: 1.9663 - val_accuracy: 0.2804
Epoch 4/10
331/331 [==============================] - 1s 3ms/step - loss: 3.0185 - accur

In [50]:
def print_expr():
    y_val_preds=model_expr.predict(X_of_expr_only_val)
    y_pred=np.argmax(y_val_preds,axis=1)
    print('Acc:',(y_pred==y_of_expr_only_val).mean(), 'F1:',f1_score(y_true=y_of_expr_only_val,y_pred=y_pred, average="macro"))
    print(metric_for_Exp(y_of_expr_only_val,y_pred))

if True:
    print_expr()
    print('Best weights:')
    model_expr.set_weights(best_model_weights)
print_expr()


Acc: 0.267445810966909 F1: 0.24190535710143465
(0.24190535710143465, 0.267445810966909, [0.24436826640548484, 0.15917525773195876, 0.2960482250502344, 0.15828677839851024, 0.39813474835182505, 0.1691542288557214, 0.20785969470607338, 0.3022156573116691])
Best weights:
Acc: 0.28035605082557585 F1: 0.2575082724838059
(0.2575082724838059, 0.28035605082557585, [0.21838407494145198, 0.16407355021216405, 0.3193987787693753, 0.17784050811573748, 0.3757660654876554, 0.247171453437772, 0.22011661807580177, 0.33731513083048914])


eyes (280):
Acc: 0.2779778487463478 F1: 0.07406295613044757
(0.07406295613044757, 0.2779778487463478, [0.0, 0.0, 0.0, 0.0, 0.005359056806002143, 0.0, 0.13456362937331795, 0.45258096286426047])

gaze (8):
Acc: 0.16056261466331453 F1: 0.11271662722681726
(0.11271662722681726, 0.16056261466331453, [0.0, 0.10371318822023047, 0.07899934167215274, 0.050259965337954945, 0.17420170723996203, 0.0851218624954529, 0.07574654042243263, 0.33369041242635245])

AU(35):
Acc: 0.29809064347353403 F1: 0.26092886553983147
(0.26092886553983147, 0.29809064347353403, [0.2355798421372192, 0.19155844155844157, 0.2949579831932773, 0.07401177460050462, 0.47602686240824615, 0.2528344671201814, 0.24332909783989834, 0.31913245546088304])

pose(6):
Acc: 0.12638445335326493 F1: 0.08603602630991644
(0.08603602630991644, 0.12638445335326493, [0.06047244094488188, 0.0257201646090535, 0.11429949360983842, 0.01775523145212429, 0.052318668252080855, 0.07452681388012618, 0.05972696245733788, 0.28346843527388854])

all features
Acc: 0.31528164707481143 F1: 0.19678978218884605
(0.19678978218884605, 0.31528164707481143, [0.050495049504950484, 0.0, 0.1168032786885246, 0.008708272859216257, 0.42180634662327093, 0.2803946530872056, 0.2761535752025361, 0.4199570815450644])

-eye
Acc: 0.28735475980159 F1: 0.23708360924334548
(0.23708360924334548, 0.28735475980159, [0.2883403361344538, 0.15083798882681565, 0.262343404568902, 0.06230769230769231, 0.4243018128368447, 0.06851119894598155, 0.2667564836645335, 0.3732699566615406])

## VA 

In [55]:
batch_size=512 #128
model_va=Sequential()
if True:
    model_va.add(Dense(2, input_shape=X_of_train.shape[1:],activation='tanh',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    model_va.add(Dense(128, input_shape=X_of_train.shape[1:],activation='relu')) #256
    model_va.add(Dense(2,activation='tanh'))

In [56]:
model_va.compile(optimizer=Adam(lr=1e-3), loss=CCC_VA, metrics=['mae','mse'])
model_va.summary()

save_best_model = SaveBestModel('val_loss',False)
model_va.fit(X_of_va_only_train,y_of_va_only_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], 
          validation_data=(X_of_va_only_val,y_of_va_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 72        
Total params: 72
Trainable params: 72
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
192/192 [==============================] - 2s 6ms/step - loss: 0.9100 - mae: 0.4381 - mse: 0.3077 - val_loss: 0.8424 - val_mae: 0.4137 - val_mse: 0.2826
Epoch 2/20
192/192 [==============================] - 1s 5ms/step - loss: 0.7758 - mae: 0.3887 - mse: 0.2444 - val_loss: 0.8003 - val_mae: 0.3777 - val_mse: 0.2407
Epoch 3/20
192/192 [==============================] - 1s 5ms/step - loss: 0.7113 - mae: 0.3512 - mse: 0.2025 - val_loss: 0.7906 - val_mae: 0.3611 - val_mse: 0.2244
Epoch 4/20
192/192 [==============================] - 1s 5ms/step - loss: 0.6843 - mae: 0.3321 - mse: 0.1834 - val_loss: 0.7881 - val_mae: 0.3530 - val_mse: 0.2

In [57]:
def print_ccc():
    y_val_preds=model_va.predict(X_of_va_only_val)
    pred_V=y_val_preds[:,0]
    pred_A=y_val_preds[:,1]
    gt_V=y_of_va_only_val[:,0]
    gt_A=y_of_va_only_val[:,1]
    print(metric_for_VA(gt_V,gt_A,pred_V,pred_A))
    print(CCC_numpy(gt_V,pred_V),CCC_numpy(gt_A,pred_A))

if True:
    print_ccc()
    print('Best weights:')
    model_va.set_weights(best_model_weights)
print_ccc()

(0.19568500673045028, 0.27970677685538065, 0.23769589179291545)
0.1956925946400336 0.27971762259751964
Best weights:
(0.1964146670054143, 0.289529188272769, 0.24297192763909164)
0.1964222709818761 0.2895404014178175


gaze (8):
(0.09609551544301502, 0.011424792370935246, 0.05376015390697513)
0.09609923428389722 0.011425235917943427

AU(35):
(0.2091367671541376, 0.2882616356667487, 0.24869920141044313)
0.20914486016768477 0.2882728101199038

pose(6):
(8.173267441818511e-06, 0.00012606241963503896, 6.711784353842874e-05)
8.173584345890638e-06 0.00012606730310202639

eyes/all:
0.0 0.0

-eyes:
(0.1900476866797427, 0.2670194001668687, 0.2285335434233057)
0.1900550573300604 0.2670297612825224

## AU 

In [58]:
batch_size=512 #128
model_au=Sequential()
if False:
    model_au.add(Dense(y_of_au_only_train.shape[1], input_shape=X_of_train.shape[1:],activation='sigmoid',use_bias=True,kernel_regularizer=tf.keras.regularizers.l2(1.0/batch_size)))
else:
    model_au.add(Dense(128, input_shape=X_of_train.shape[1:],activation='relu')) #256
    model_au.add(Dense(y_of_au_only_train.shape[1],activation='sigmoid'))

In [59]:
model_au.compile(optimizer=Adam(lr=1e-3), loss=loss_aus, metrics=metrics)
model_au.summary()

save_best_model = SaveBestModel('val_loss',False)
model_au.fit(X_of_au_only_train,y_of_au_only_train, batch_size=batch_size, epochs=20, verbose=1, callbacks=[save_best_model], 
          validation_data=(X_of_au_only_val,y_of_au_only_val))
best_model_weights = save_best_model.best_model_weights
print(save_best_model.best)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 128)               4608      
_________________________________________________________________
dense_12 (Dense)             (None, 12)                1548      
Total params: 6,156
Trainable params: 6,156
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
191/191 [==============================] - 2s 8ms/step - loss: 0.6422 - auc: 0.6694 - binary_accuracy: 0.6173 - recall_1: 0.6387 - precision_1: 0.3247 - val_loss: 0.6407 - val_auc: 0.6982 - val_binary_accuracy: 0.6790 - val_recall_1: 0.6056 - val_precision_1: 0.4095
Epoch 2/20
191/191 [==============================] - 1s 7ms/step - loss: 0.5669 - auc: 0.7702 - binary_accuracy: 0.7126 - recall_1: 0.6479 - precision_1: 0.4083 - val_loss: 0.6484 - val_auc: 0.6984 - val_binary_accuracy: 0.6943 - val_r

In [60]:
def print_au():
    y_val_preds=model_au.predict(X_of_au_only_val)
    new_pred = ((y_val_preds >= 0.5) * 1)
    print(np.mean([f1_score(y_true=y_of_au_only_val[:,i],y_pred=new_pred[:,i]) for i in range(y_val_preds.shape[1])]))
    print(f1_score_max(y_of_au_only_val,y_val_preds,thresh=np.arange(0.1,1,0.1)))

print_au()
if True:
    print('Best weights:')
    model_au.set_weights(best_model_weights)
    print_au()

0.39647167474999107
(0.4220559777042339, 0.5083333333333333, array([0.45014739, 0.33946897, 0.39683331, 0.5076999 , 0.6861697 ,
       0.65142776, 0.58635382, 0.08401254, 0.07576493, 0.13638968,
       0.86687881, 0.2835249 ]), array([0.6, 0.7, 0.4, 0.4, 0.3, 0.3, 0.5, 0.6, 0.7, 0.8, 0.1, 0.7]), array([0.75414793, 0.84280509, 0.62490309, 0.61335091, 0.61990231,
       0.61811909, 0.72526748, 0.83008994, 0.85245775, 0.94158009,
       0.76647542, 0.81877035]))
Best weights:
0.402943536240024


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4285070719228661, 0.49166666666666653, array([0.47324988, 0.35777518, 0.38066586, 0.53532446, 0.68493814,
       0.64668806, 0.59833856, 0.09717257, 0.07920792, 0.12415086,
       0.86703449, 0.29753888]), array([0.6, 0.7, 0.4, 0.5, 0.3, 0.4, 0.5, 0.6, 0.6, 0.6, 0.1, 0.6]), array([0.77977206, 0.85094588, 0.52116607, 0.6682819 , 0.59625523,
       0.6457978 , 0.71697162, 0.78461777, 0.75484571, 0.85505505,
       0.76550628, 0.77760118]))


gaze (8):
0.33700727040525874
(0.37684445283245993, 0.375, array([0.3462248 , 0.22813971, 0.33352061, 0.50523492, 0.66614959,
       0.60036353, 0.49285385, 0.09313233, 0.0717555 , 0.0844358 ,
       0.86675746, 0.23356535]), array([0.3, 0.5, 0.3, 0.4, 0.1, 0.1, 0.5, 0.5, 0.6, 0.6, 0.1, 0.5]), array([0.24441774, 0.75841216, 0.21821213, 0.41378508, 0.49941851,
       0.42894247, 0.48829276, 0.68518375, 0.75628004, 0.81756862,
       0.76484726, 0.69266553]))

eyes:
0.3564600478556352
(0.37960306981121555, 0.4083333333333334, array([0.36417588, 0.24914956, 0.33002594, 0.49272711, 0.67171943,
       0.6007334 , 0.51602659, 0.09207805, 0.05743645, 0.05730099,
       0.86672671, 0.25713672]), array([0.8, 0.6, 0.1, 0.3, 0.5, 0.3, 0.6, 0.4, 0.2, 0.1, 0.3, 0.7]), array([0.72588773, 0.75186075, 0.1990231 , 0.33350132, 0.54496821,
       0.43018297, 0.49193673, 0.60858273, 0.04702279, 0.10203132,
       0.7648085 , 0.73166382]))

AUs(35):
0.40392001046178416
(0.42757883295563887, 0.5166666666666666, array([0.4655188 , 0.35727045, 0.38784565, 0.53174125, 0.68184941,
       0.64406905, 0.59717051, 0.09542041, 0.07538192, 0.12864622,
       0.86645997, 0.29957236]), array([0.6, 0.7, 0.4, 0.5, 0.4, 0.4, 0.6, 0.6, 0.6, 0.7, 0.1, 0.6]), array([0.77015816, 0.86052101, 0.5400062 , 0.66859203, 0.64095209,
       0.62591099, 0.74833307, 0.7618623 , 0.71375407, 0.90967592,
       0.7645759 , 0.75872228]))

pose(6):
0.3182604488687078
(0.36707807907659173, 0.35833333333333334, array([0.34642003, 0.20899408, 0.33209402, 0.49288198, 0.6663384 ,
       0.6003961 , 0.49011684, 0.08241402, 0.05686594, 0.06029628,
       0.8667765 , 0.20134275]), array([0.5, 0.5, 0.1, 0.4, 0.4, 0.2, 0.4, 0.7, 0.2, 0.5, 0.1, 0.3]), array([0.59730191, 0.74089006, 0.1991394 , 0.33854086, 0.50089161,
       0.42902   , 0.3690107 , 0.7807412 , 0.03430764, 0.57950845,
       0.76488603, 0.11459141]))

all features
0.36418628630291255
(0.4300552415494017, 0.47500000000000003, array([0.45422924, 0.35724585, 0.39790889, 0.53198421, 0.69472495,
       0.65344145, 0.60027301, 0.09012674, 0.09273248, 0.10274914,
       0.86794033, 0.31730661]), array([0.7, 0.8, 0.2, 0.5, 0.4, 0.3, 0.3, 0.7, 0.6, 0.2, 0.1, 0.9]), array([0.75062025, 0.86176151, 0.68747093, 0.63242363, 0.64374322,
       0.64378198, 0.73891301, 0.77457745, 0.86497907, 0.79756551,
       0.76767716, 0.76496356]))
-eye
0.4022962580787666
(0.4261672537136978, 0.5083333333333333, array([0.47709188, 0.35737077, 0.33555226, 0.5395627 , 0.69240549,
       0.65095143, 0.59304883, 0.09115419, 0.06979158, 0.14028469,
       0.86769587, 0.29909734]), array([0.6, 0.7, 0.4, 0.5, 0.4, 0.4, 0.5, 0.7, 0.5, 0.6, 0.3, 0.5]), array([0.75798573, 0.84385176, 0.39250271, 0.63591254, 0.64358815,
       0.61885564, 0.71313382, 0.72557761, 0.54841836, 0.80566755,
       0.76977051, 0.77725229]))

In [51]:
print('\nExpression')
y_of_val_preds_expr=model_expr.predict(X_of_val_all)
success_expr=(masks_val_success[masks_of_expr_val_all==1]==1)
y_pred=np.argmax(y_of_val_preds_expr,axis=1)
print((y_pred==y_of_expr_val_all)[masks_of_expr_val_all==1][success_expr].mean())
f1_expr=f1_score(y_true=y_of_expr_val_all[masks_of_expr_val_all==1][success_expr],
                 y_pred=y_pred[masks_of_expr_val_all==1][success_expr], average="macro")
print(f1_expr)
print(metric_for_Exp(y_of_expr_val_all[masks_of_expr_val_all==1][success_expr],y_pred[masks_of_expr_val_all==1][success_expr]))



Expression
0.28035605082557585
0.2575082724838059
(0.2575082724838059, 0.28035605082557585, [0.21838407494145198, 0.16407355021216405, 0.3193987787693753, 0.17784050811573748, 0.3757660654876554, 0.247171453437772, 0.22011661807580177, 0.33731513083048914])


In [107]:
print('\nVA')
y_of_val_preds_va=model_va.predict(X_of_val_all)
success_va=(masks_val_success[masks_of_va_val_all==1]==1)
pred_V=y_of_val_preds_va[masks_of_va_val_all==1,0][success_va]
pred_A=y_of_val_preds_va[masks_of_va_val_all==1,1][success_va]
gt_V=y_of_va_val_all[masks_of_va_val_all==1,0][success_va]
gt_A=y_of_va_val_all[masks_of_va_val_all==1,1][success_va]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nAUs')
y_of_val_preds_aus=model_au.predict(X_of_val_all)
success_aus=(masks_val_success[masks_of_aus_val_all==1]==1)
new_pred = ((y_of_val_preds_aus >= 0.5) * 1)
f1_au=np.mean([f1_score(y_true=y_of_aus_val_all[masks_of_aus_val_all==1,i][success_aus],
                        y_pred=new_pred[masks_of_aus_val_all==1,i][success_aus]) for i in range(y_of_val_preds_aus.shape[1])])
print(f1_au)
print(f1_score_max(y_of_aus_val_all[masks_of_aus_val_all==1][success_aus],
                   y_of_val_preds_aus[masks_of_aus_val_all==1][success_aus],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)
#Total 0.2419729432148961 0.2559140140366414 0.4024556551490766 0.9003426124006141


VA
(25796,) 0.19641466700852012 0.2895291882595464 0.24297192763403325

AUs
0.402943536240024


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4285070719228661, 0.49166666666666653, array([0.47324988, 0.35777518, 0.38066586, 0.53532446, 0.68493814,
       0.64668806, 0.59833856, 0.09717257, 0.07920792, 0.12415086,
       0.86703449, 0.29753888]), array([0.6, 0.7, 0.4, 0.5, 0.3, 0.4, 0.5, 0.6, 0.6, 0.6, 0.1, 0.6]), array([0.77977206, 0.85094588, 0.52116607, 0.6682819 , 0.59625523,
       0.6457978 , 0.71697162, 0.78461777, 0.75484571, 0.85505505,
       0.76550628, 0.77760118]))

Total 0.24297192763403325 0.2571850139826878 0.402943536240024 0.9031004778567451


## Ensemble 

In [52]:
print(y_of_expr_val_all.shape,y_of_va_val_all.shape,y_of_aus_val_all.shape)
print(y_pred_expr.shape,y_pred_va.shape,y_pred_aus.shape)

(26876,) (26876, 2) (26876, 12)
(51159, 8) (51159, 2) (51159, 12)


In [53]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_val)
print('\nVA')
gt_V=y_va_val[masks_va_val==1,0]
gt_A=y_va_val[masks_va_val==1,1]
pred_V=y_pred_va[masks_va_val==1,0]
pred_A=y_pred_va[masks_va_val==1,1]
ccc_V,ccc_A,ccc_VA=metric_for_VA(gt_V,gt_A,pred_V,pred_A)
print(gt_V.shape,ccc_V,ccc_A,ccc_VA)

print('\nExpression')
print(y_expr_val[masks_expr_val==1].shape)
y_pred=np.argmax(y_pred_expr,axis=1)
print((y_pred==y_expr_val)[masks_expr_val==1].mean())
f1_expr=f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro")
print(f1_expr)
print(metric_for_Exp(y_expr_val[masks_expr_val==1],y_pred[masks_expr_val==1]))

print('\nAUs')
new_pred = ((y_pred_aus >= 0.5) * 1)
print(new_pred[masks_aus_val==1,:].shape)
f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_pred_aus.shape[1])])
print(f1_au)
print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_pred_aus[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)))

total=ccc_VA+f1_expr+f1_au
print('\nTotal',ccc_VA,f1_expr,f1_au,total)


VA
(26876,) 0.4749176554355453 0.41918481366710747 0.4470512345513264

Expression
(15440,)
0.35174870466321245
0.33507035885799447
(0.33507035885799447, 0.35174870466321245, [0.3309143686502177, 0.21268097853220172, 0.4306987399770905, 0.2674139311449159, 0.4919247295895688, 0.45335129673290675, 0.17721518987341772, 0.31636363636363635])

AUs
(26876, 12)
0.4943377041019861
(0.5217870390537378, 0.5583333333333335, array([0.58350767, 0.46639283, 0.58247955, 0.60487214, 0.73661289,
       0.71948443, 0.67720908, 0.25145518, 0.18346774, 0.2108389 ,
       0.88044481, 0.36467925]), array([0.6, 0.7, 0.5, 0.3, 0.4, 0.4, 0.5, 0.9, 0.8, 0.7, 0.2, 0.7]), array([0.82222057, 0.89365977, 0.80439798, 0.68436523, 0.70296919,
       0.72953565, 0.80101206, 0.95215062, 0.93972317, 0.92089597,
       0.80238875, 0.84339187]))

Total 0.4470512345513264 0.33507035885799447 0.4943377041019861 1.276459297511307


In [54]:
for w in np.linspace(0,1,11):
    y_ensemble=w*y_pred_expr+(1-w)*y_of_val_preds_expr*masks_val_success.reshape((-1,1))
    y_pred=np.argmax(y_ensemble,axis=1)
    print(w,'F1:',f1_score(y_true=y_expr_val[masks_expr_val==1],y_pred=y_pred[masks_expr_val==1], average="macro"))
    #print(metric_for_Exp(gt_expr,y_pred))

0.0 F1: 0.25134282824069276
0.1 F1: 0.2719023289473588
0.2 F1: 0.2900276546934521
0.30000000000000004 F1: 0.31073224086083157
0.4 F1: 0.3337807788288267
0.5 F1: 0.34577055619286023
0.6000000000000001 F1: 0.3542082155899847
0.7000000000000001 F1: 0.3529213573033453
0.8 F1: 0.3475036965122771
0.9 F1: 0.34137692566502637
1.0 F1: 0.33507035885799447


In [66]:
for w in np.linspace(0,1,11):
    y_ensemble=w*y_pred_va+(1-w)*y_of_val_preds_va*masks_val_success.reshape((-1,1))
    pred_V=y_ensemble[masks_va_val==1,0]
    pred_A=y_ensemble[masks_va_val==1,1]
    print(w,metric_for_VA(gt_V,gt_A,pred_V,pred_A))

0.0 (0.19709510723522208, 0.2679740589847627, 0.23253458310999237)
0.1 (0.2379344957539207, 0.2953908004476591, 0.2666626481007899)
0.2 (0.2792266606075017, 0.32254497367625407, 0.3008858171418779)
0.30000000000000004 (0.31955003055339887, 0.3484370566448255, 0.33399354359911215)
0.4 (0.35738971557552646, 0.3719672957954081, 0.3646785056854673)
0.5 (0.39130832435371604, 0.3920499644862386, 0.3916791444199773)
0.6000000000000001 (0.42011955414681934, 0.40775011202467965, 0.4139348330857495)
0.7000000000000001 (0.44302349453844625, 0.418412368328051, 0.43071793143324866)
0.8 (0.45967155771334955, 0.4237480756390997, 0.44170981667622466)
0.9 (0.4701531162154282, 0.42385745288521887, 0.4470052845503235)
1.0 (0.4749176554355453, 0.41918481366710747, 0.4470512345513264)


In [67]:
for w in np.linspace(0,1,11):
    y_ensemble=w*y_pred_aus+(1-w)*y_of_val_preds_aus*masks_val_success.reshape((-1,1))
    new_pred = ((y_ensemble >= 0.5) * 1)
    f1_au=np.mean([f1_score(y_true=y_aus_val[masks_aus_val==1,i],y_pred=new_pred[masks_aus_val==1,i]) for i in range(y_ensemble.shape[1])])
    print(w,f1_au)
    print(f1_score_max(y_aus_val[masks_aus_val==1,:],y_ensemble[masks_aus_val==1,:],thresh=np.arange(0.1,1,0.1)),'\n')

0.0 0.3971496320690291


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.42292530848517046, 0.49166666666666653, array([0.46353646, 0.3549884 , 0.3749321 , 0.52413866, 0.67794479,
       0.6379067 , 0.59227795, 0.09712522, 0.07913878, 0.12354312,
       0.85395107, 0.29562044]), array([0.6, 0.7, 0.4, 0.5, 0.3, 0.4, 0.5, 0.6, 0.6, 0.6, 0.1, 0.6]), array([0.78021283, 0.85518678, 0.52905938, 0.66698914, 0.59979164,
       0.64678524, 0.72142432, 0.79316118, 0.76447388, 0.86009823,
       0.74899539, 0.78456616])) 

0.1 0.4160251550983128


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.4406446590884676, 0.5166666666666666, array([0.4819551 , 0.37936652, 0.42602524, 0.53284592, 0.69070407,
       0.65544872, 0.6073531 , 0.10666667, 0.09787088, 0.14114441,
       0.85532522, 0.31303007]), array([0.6, 0.7, 0.5, 0.5, 0.4, 0.4, 0.5, 0.6, 0.6, 0.6, 0.2, 0.6]), array([0.7965099 , 0.87241405, 0.72920077, 0.68142581, 0.66933323,
       0.6640125 , 0.73694002, 0.80555142, 0.8045096 , 0.88272064,
       0.75461378, 0.80108647])) 

0.2 0.43793591072827615


...

(0.46096259292810443, 0.49166666666666653, array([0.50423925, 0.39517228, 0.49463627, 0.54864449, 0.70966629,
       0.66905679, 0.62069356, 0.11856418, 0.11599006, 0.1680831 ,
       0.86067095, 0.32613391]), array([0.5, 0.7, 0.5, 0.4, 0.4, 0.4, 0.5, 0.6, 0.6, 0.6, 0.1, 0.6]), array([0.76067867, 0.884395  , 0.77563626, 0.64008781, 0.68696979,
       0.6771469 , 0.7509302 , 0.8190951 , 0.84119661, 0.90165947,
       0.75692067, 0.81425807])) 

0.30000000000000004 0.4590171416311466


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.47987252887762133, 0.5249999999999999, array([0.53451692, 0.41157025, 0.55286983, 0.56354172, 0.72101833,
       0.68479009, 0.63436008, 0.13264059, 0.13313462, 0.18889282,
       0.86509932, 0.33603578]), array([0.5, 0.7, 0.5, 0.4, 0.4, 0.4, 0.6, 0.7, 0.6, 0.6, 0.3, 0.6]), array([0.7814779 , 0.89403185, 0.81072332, 0.65995684, 0.69582527,
       0.69325793, 0.79197053, 0.89440393, 0.87014437, 0.91468224,
       0.7824453 , 0.82326239])) 

0.4 0.47809381580087645


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.49492153865819516, 0.5083333333333332, array([0.56770456, 0.42424242, 0.5847769 , 0.57564178, 0.72828696,
       0.69915773, 0.65136484, 0.14442013, 0.1502079 , 0.19978441,
       0.8708717 , 0.34259913]), array([0.5, 0.6, 0.5, 0.4, 0.4, 0.4, 0.5, 0.7, 0.6, 0.6, 0.3, 0.6]), array([0.80008186, 0.86709332, 0.82341122, 0.67770502, 0.70095996,
       0.70762018, 0.77902218, 0.89816193, 0.87833011, 0.91713797,
       0.7918217 , 0.82664831])) 

0.5 0.48823751449356695
(0.5047758027378862, 0.5083333333333332, array([0.58386993, 0.44379467, 0.58903171, 0.5884654 , 0.73517362,
       0.71011167, 0.66320811, 0.1621955 , 0.15655676, 0.20485744,
       0.87313433, 0.3469105 ]), array([0.5, 0.6, 0.5, 0.4, 0.4, 0.4, 0.5, 0.7, 0.6, 0.6, 0.3, 0.6]), array([0.80763506, 0.87260009, 0.82155083, 0.69467183, 0.70657836,
       0.71796398, 0.78984968, 0.89890609, 0.87531627, 0.91594731,
       0.79505879, 0.82460188])) 

0.6000000000000001 0.49432048663609424
(0.5137311350717509, 0.5333333333333333, arr

# Save test results

In [49]:
with open(os.path.join(DATA_DIR,'MTL_Challenge_test_set_release.txt'),'r') as f:
    test_set_images=f.read().splitlines()
print(len(test_set_images),test_set_images[:5])

51160 ['image,valence,arousal,expression,aus', 'video45_1/00001.jpg', 'video45_1/00008.jpg', 'video45_1/00017.jpg', 'video45_1/00021.jpg']


In [50]:
resdir=DATA_DIR /'test_results/MTL'
if not os.path.exists(resdir):
    os.makedirs(resdir)
header = 'image,valence,arousal,expression,aus\n'

In [51]:
num_missed=0
X_test=[]
for imagename in test_set_images[1:]:
    if imagename in filename2featuresTest:
        if FEATURES_SCORES_AGGREGATION==FEATURES_ONLY:
            X_test.append(filename2featuresTest[imagename][0])
        elif FEATURES_SCORES_AGGREGATION==SCORES_ONLY:
            X_test.append(filename2featuresTest[imagename][1])
        else:
            X_test.append(np.concatenate((filename2featuresTest[imagename][0],filename2featuresTest[imagename][1])))
    else:
        num_missed+=1
    #elif (valence==-5 or arousal==-5) and valence!=arousal:
    #    print(valence,arousal)
X_test=np.array(X_test)
print(X_test.shape,num_missed)

(51159, 1290) 0


In [52]:
y_pred_expr,y_pred_va,y_pred_aus=mtl_model_new.predict(X_test)
pred_V=y_pred_va[:,0]
pred_A=y_pred_va[:,1]
print('VA',pred_V.shape,pred_A.shape,pred_V[:5],pred_A[:5])

y_pred=np.argmax(y_pred_expr,axis=1)
print('Expression',y_expr_val.shape,y_pred.shape,y_pred[:5])

thresholds=0.5
aus_pred = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred.shape,aus_pred[:5])

thresholds=np.array([0.6, 0.7, 0.5, 0.3, 0.4, 0.4, 0.5, 0.9, 0.8, 0.7, 0.2, 0.7])
aus_pred_dynamic = ((y_pred_aus >= thresholds) * 1)
print('AUs',y_pred_aus.shape,aus_pred_dynamic.shape,aus_pred_dynamic[:5])

VA (51159,) (51159,) [0.7118035  0.49976918 0.55337983 0.50104845 0.42740974] [0.31474793 0.36706084 0.37494317 0.43688834 0.5564812 ]
Expression (26876,) (51159,) [7 5 4 4 7]
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]]
AUs (51159, 12) (51159, 12) [[0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 1 0 0 0 1 0]]


In [53]:
with open(os.path.join(resdir,'1_mtl_enet0_slice_aus_0.5.txt'), 'w') as f:
#with open(os.path.join(resdir,'3_mtl_enet0_slice_aus_0.5_all.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

In [33]:
with open(os.path.join(resdir,'2_mtl_enet0_slice_aus_dyn.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred_dynamic):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)

## OpenFace 

In [63]:
num_missed=num_fails=0
X_openface_test=[]
mask_success=[]

prev_videoname=''
video_frames=None

for imagename in test_set_images[1:]:
    videoname,filename=imagename.split('/')
    if videoname!=prev_videoname:
        fpath=test_outdir / (videoname+'.csv')
        if os.path.exists(fpath):
            df = pd.read_csv(fpath)
            success=(df.success==1).to_numpy()
            df = df.drop(columns=['frame', 'face_id','timestamp', 'confidence', 'success'])
            df=df.drop(columns=[c for c in df.columns if c.startswith('eye')])
            df=df.drop(columns=[c for c in df.columns if c.startswith('gaze')])
            #df=df.drop(columns=[c for c in df.columns if c.startswith('AU')])
            df=df.drop(columns=[c for c in df.columns if c.startswith('pose')])
            cur_features=df.to_numpy()

            video_frames={f:i for i,f in enumerate(sorted(os.listdir(test_data_dir / videoname)))}
            #print(videoname,cur_features.shape,len(video_frames))
            prev_videoname=videoname

    if video_frames is not None and filename in video_frames:
        ind=video_frames[filename]
        mask_success.append(success[ind])
        X_openface_test.append(cur_features[ind])
        if not success[ind]:
            num_fails+=1
    else:
        num_missed+=1
X_openface_test=np.array(X_openface_test)
mask_success=np.array(mask_success)
print(X_openface_test.shape,mask_success.shape,num_missed,mask_success.sum(),num_fails)

(51159, 35) (51159,) 0 47790 3369


In [65]:
y_of_test_preds_expr=model_expr.predict(X_openface_test)
w=0.6
y_ensemble=w*y_pred_expr+(1-w)*y_of_test_preds_expr*mask_success.reshape((-1,1))
y_pred=np.argmax(y_ensemble,axis=1)
with open(os.path.join(resdir,'4_mtl_enet0_slice_aus_0.5_of.txt'), 'w') as f:
    f.write(header)
    for filename, v,a,expr,aus in zip(test_set_images[1:],pred_V,pred_A,y_pred,aus_pred):
        s=','.join([filename,str(v),str(a),str(expr),*map(str,aus)])+'\n'
        f.write(s)